In [ ]:
import cupy
def process_0():
    import cupyx.distributed
    cupy.cuda.Device(0).use()
    comm = cupyx.distributed.init_process_group(2, 0)
    array = cupy.ones(1)
    comm.broadcast(array, 0)

def process_1():
    import cupyx.distributed
    cupy.cuda.Device(1).use()
    comm = cupyx.distributed.init_process_group(2, 1)
    array = cupy.zeros(1)
    comm.broadcast(array, 0)
    cupy.equal(array, cupy.ones(1))

In [ ]:
process_0()

In [ ]:
2**(2*8)

In [ ]:
import numpy as np
a=np.array([2,4,6,3,5])
print(np.argsort(a))
a=a[[1,0,2,3,4]]
print(a)
np.argsort(a)

In [ ]:
def calc_J_inv(I):
    '''
    input:
    I:mutual information
    output:
    gamma:channel SNR Es/No
    ----
    referrence:
    POLAR CODES FOR ERROR CORRECTION:
    ANALYSIS AND DECODING ALGORITHMS
    p37
    (4.5)
    '''
    if I>1 or I<0:
        print("I is err")
    
    a1=1.09542
    b1=0.214217
    c1=2.33727
    a2=0.706692
    b2=0.386013
    c2=-1.75017
    I_thresh=0.3646
    
    if I<I_thresh:
        sigma=a1*I**2+b1*I+c1*I**(1/2)
    else:
        sigma=-a2*np.log(b2*(1-I))-c2*I
        
    gamma=sigma**2/8
    #gamma_dB=10*math.log10(gamma)
    return gamma

In [1]:
import numpy as np
import matplotlib.pyplot as plt
y=np.arange(0,1,0.03)
x=np.zeros(len(y))
for i in range(len(y)):
    x[i]=calc_J_inv(y[i])
    
x=10*np.log10(x)
plt.plot(x,y)

filename="J_inverse_function"
with open(filename,'w') as f:
    for i in range(len(x)):
        print(str(x[i]),str(y[i]),file=f)



NameError: name 'calc_J_inv' is not defined

In [24]:
def reverse(index,n):
    '''
    make n into bit reversal order
    '''
    tmp=format (index,'b')
    tmp=tmp.zfill(n+1)[:0:-1]
    res=int(tmp,2) 
    return res

In [45]:
import numpy as np
N=1024
K=512
M=256
EsNodB=9.0
dmin=(6/(M-1))**(1/2)
gamma=10**(EsNodB/10)*dmin/2

gammadb=10*np.log10(gamma)
print(gammadb)

-2.1522446068913688


In [48]:

n=int(np.log2(N))

filename=str(-2.2)+'.dat'
print(filename)
a=np.loadtxt(filename,dtype=str)
xi=np.zeros(N)
print(a)
xi=a[:,1].astype(float)

-2.2.dat
[['0,' '-1.988294e+03']
 ['1,' '-3.986529e+02']
 ['2,' '-5.493686e+02']
 ...
 ['1021,' '2.283489e+01']
 ['1022,' '2.157829e+01']
 ['1023,' '2.795075e+01']]


In [49]:
tmp=np.argsort(xi)
frozen_bits=np.sort(tmp[:N-K])
info_bits=np.sort(tmp[N-K:])
#bit reversal order
for i in range(len(frozen_bits)):
    frozen_bits[i]=reverse(frozen_bits[i],n)
    
frozen_bits=np.sort(frozen_bits)
    
for i in range(len(info_bits)):
    info_bits[i]=reverse(info_bits[i],n)
info_bits=np.sort(info_bits)


In [50]:
from polar_code.RCA import RCA
const=RCA()
a1,b1=const.main_const(N,K,EsNodB,256)
a,b=const.main_const(N,K,gammadb,2)
print(np.sum(a1!=a))


dmin=(6/(M-1))**(1/2)
gamma=10**(EsNodB/10)*dmin/2 #QPSK(sqrt(2))
#gamma=gamma*np.ones(N)


0


In [51]:
from polar_code.iGA import Improved_GA

const=Improved_GA()
c,d=const.main_const(N,K,EsNodB,256)

In [52]:
print(np.sum(a!=frozen_bits))
print(np.sum(c!=frozen_bits))
print(np.sum(c!=a))
#print(a)
#print(frozen_bits)

0
277
277


In [57]:
M=256
import math
for i in range(0,int(math.log2(M)))[::-1]:
    print(i)

7
6
5
4
3
2
1
0


In [54]:
const=0
for i in range(len(a)):
    if np.any(a[i]==frozen_bits)==False:
        const+=1

print(const)

const=0
for i in range(len(c)):
    if np.any(c[i]==frozen_bits)==False:
        const+=1

print(const)


const=0
for i in range(len(c)):
    if np.any(c[i]==a)==False:
        const+=1

print(const)

0
2
2
